In [6]:
# Install AMPL Python API
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.3 MB/s eta 0:00:00


In [7]:
from amplpy import AMPL, ampl_notebook

# Initialize AMPL with solvers for Colab environment
# Get your own free license at: https://ampl.com/ce or https://ampl.com/courses
ampl_notebook(
    modules=["highs"],  # Open-source solver; add "gurobi", "cplex" if needed
    license_uuid="40a0a160-668d-492d-be4a-236079be8ff7"
)

print("✓ AMPL environment ready!")

Licensed to Bundle #7384.7942 expiring 20260531: ISYE 671 linear optimization and network flows, Prof. Ziteng Wang, Northern Illinois University.
✓ AMPL environment ready!


PART 1

Exercise 1 — Expanded Resource Allocation

In [8]:
%%writefile resource_ex1.mod
# ===========================================
# Exercise 1: Expanded Resource Allocation
# ===========================================

# --- SETS ---
set PRODUCTS;
set RESOURCES;

# --- PARAMETERS ---
param profit {PRODUCTS};
param usage {RESOURCES, PRODUCTS};
param available {RESOURCES};

# --- DECISION VARIABLES ---
var Produce {p in PRODUCTS} >= 0;

# --- OBJECTIVE ---
maximize Total_Profit:
    sum {p in PRODUCTS} profit[p] * Produce[p];

# --- CONSTRAINTS ---
subject to Resource_Limit {r in RESOURCES}:
    sum {p in PRODUCTS} usage[r,p] * Produce[p] <= available[r];

Overwriting resource_ex1.mod


In [9]:
%%writefile resource_ex1.dat
# ===========================================
# Data: Expanded Furniture Production
# ===========================================

set PRODUCTS := Table Chair Desk Bookcase;
set RESOURCES := Labor Wood Machine;

param profit :=
  Table     70
  Chair     50
  Desk      90
  Bookcase  80;

param usage:        Table  Chair  Desk  Bookcase :=
  Labor              4      3      5       4
  Wood              30     20     40      35
  Machine          2.0    1.5    3.0     2.5;

param available :=
  Labor    400
  Wood     2000
  Machine  250;

Overwriting resource_ex1.dat


In [10]:
from amplpy import AMPL

ex1 = AMPL()
ex1.read("resource_ex1.mod")
ex1.read_data("resource_ex1.dat")

ex1.option["solver"] = "highs"
ex1.solve()

print("Solver Status:", ex1.get_value("solve_result"))
print()

# Objective
z = ex1.obj["Total_Profit"].value()
print(f"Maximum Profit: ${z:,.2f}\n")

# Decision variables
print("Optimal Production Plan:")
Produce = ex1.var["Produce"]
for p in ex1.set["PRODUCTS"].members():
    print(f"  {p}: {Produce[p].value():.1f} units")

print("\nResource Analysis:")
print("=" * 55)

Resource_Limit = ex1.con["Resource_Limit"]
available = ex1.param["available"]

for r in ex1.set["RESOURCES"].members():
    used = Resource_Limit[r].body()
    avail = float(available[r])
    slack = avail - used
    dual = Resource_Limit[r].dual()

    print(f"\n{r}:")
    print(f"  Used: {used:.1f} / {avail:.1f} (Slack: {slack:.1f})")
    print(f"  Shadow Price: ${dual:.2f}")
    if dual > 1e-6:
        print(f"  → Binding! Each extra unit worth ${dual:.2f}")
    else:
        print("  → Non-binding (slack available)")

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 5000
3 simplex iterations
0 barrier iterations
Solver Status: solved

Maximum Profit: $5,000.00

Optimal Production Plan:
  Table: 0.0 units
  Chair: 100.0 units
  Desk: 0.0 units
  Bookcase: 0.0 units

Resource Analysis:

Labor:
  Used: 300.0 / 400.0 (Slack: 100.0)
  Shadow Price: $0.00
  → Non-binding (slack available)

Wood:
  Used: 2000.0 / 2000.0 (Slack: 0.0)
  Shadow Price: $2.50
  → Binding! Each extra unit worth $2.50

Machine:
  Used: 150.0 / 250.0 (Slack: 100.0)
  Shadow Price: $0.00
  → Non-binding (slack available)


Exercise 2 — Diet Problem

In [11]:
%%writefile diet_ex2.mod
# ===========================================
# Exercise 2: Diet Problem
# ===========================================

set FOODS;
set NUTRIENTS;

param cost {FOODS} >= 0;
param max_serv {FOODS} >= 0;

param amt {NUTRIENTS, FOODS} >= 0;

param min_req {NUTRIENTS} default 0;
param max_req {NUTRIENTS} default Infinity;

var Servings {f in FOODS} >= 0, <= max_serv[f];

minimize Total_Cost:
    sum {f in FOODS} cost[f] * Servings[f];

subject to Min_Req {n in NUTRIENTS}:
    sum {f in FOODS} amt[n,f] * Servings[f] >= min_req[n];

subject to Max_Req {n in NUTRIENTS}:
    sum {f in FOODS} amt[n,f] * Servings[f] <= max_req[n];

Writing diet_ex2.mod


In [12]:
%%writefile diet_ex2.dat
# ===========================================
# Data: Diet Problem
# ===========================================

set FOODS := Oatmeal Chicken Rice Broccoli Milk;
set NUTRIENTS := Calories Protein Fat Carbs;

param cost :=
  Oatmeal   0.50
  Chicken   2.50
  Rice      0.30
  Broccoli  1.00
  Milk      0.80;

param max_serv :=
  Oatmeal   4
  Chicken   3
  Rice      5
  Broccoli  4
  Milk      3;

param amt:
            Oatmeal  Chicken  Rice  Broccoli  Milk :=
Calories       150      250    200      50     120
Protein          5       30      4       4       8
Fat              2        8    0.5     0.5       5
Carbs           27        0     45      10      12;

param min_req :=
  Calories  2000
  Protein     60
  Carbs      250;

param max_req :=
  Calories  2500
  Fat         65
  Carbs      350;

Writing diet_ex2.dat


In [13]:
from amplpy import AMPL

ex2 = AMPL()
ex2.read("diet_ex2.mod")
ex2.read_data("diet_ex2.dat")

ex2.option["solver"] = "highs"
ex2.solve()

print("Solver Status:", ex2.get_value("solve_result"))
print()

# Objective
z = ex2.obj["Total_Cost"].value()
print(f"Minimum Cost: ${z:,.4f}\n")

# Servings
print("Optimal Meal Plan (Servings):")
Servings = ex2.var["Servings"]
for f in ex2.set["FOODS"].members():
    print(f"  {f}: {Servings[f].value():.6f}")

# Nutrient totals + constraint duals
print("\nNutrient Constraints Analysis:")
print("=" * 55)

Min_Req = ex2.con["Min_Req"]
Max_Req = ex2.con["Max_Req"]

for n in ex2.set["NUTRIENTS"].members():
    min_dual = Min_Req[n].dual()
    max_dual = Max_Req[n].dual()

    # LHS values can be obtained via constraint body()
    lhs_min = Min_Req[n].body()
    lhs_max = Max_Req[n].body()

    print(f"\n{n}:")
    print(f"  Intake: {lhs_min:.4f}")
    print(f"  Dual (Min_Req): {min_dual:.6f}")
    print(f"  Dual (Max_Req): {max_dual:.6f}")

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 6.933333333
2 simplex iterations
0 barrier iterations
Solver Status: solved

Minimum Cost: $6.9333

Optimal Meal Plan (Servings):
  Oatmeal: 4.000000
  Chicken: 0.920000
  Rice: 5.000000
  Broccoli: 0.000000
  Milk: 1.416667

Nutrient Constraints Analysis:

Calories:
  Intake: 2000.0000
  Dual (Min_Req): 0.010000
  Dual (Max_Req): 0.000000

Protein:
  Intake: 78.9333
  Dual (Min_Req): 0.000000
  Dual (Max_Req): 0.000000

Fat:
  Intake: 24.9433
  Dual (Min_Req): 0.000000
  Dual (Max_Req): 0.000000

Carbs:
  Intake: 350.0000
  Dual (Min_Req): 0.000000
  Dual (Max_Req): -0.033333


Exercise 3 — Transportation


In [14]:
%%writefile transport_ex3.mod
# ===========================================
# Exercise 3: Transportation Problem
# ===========================================

set WAREHOUSES;
set CUSTOMERS;

param supply {WAREHOUSES} >= 0;
param demand {CUSTOMERS} >= 0;
param cost   {WAREHOUSES, CUSTOMERS} >= 0;

var Ship {WAREHOUSES, CUSTOMERS} >= 0;

minimize Total_Shipping_Cost:
    sum {w in WAREHOUSES, c in CUSTOMERS} cost[w,c] * Ship[w,c];

subject to Supply_Limits {w in WAREHOUSES}:
    sum {c in CUSTOMERS} Ship[w,c] <= supply[w];

subject to Meet_Demand {c in CUSTOMERS}:
    sum {w in WAREHOUSES} Ship[w,c] = demand[c];

Writing transport_ex3.mod


In [15]:
%%writefile transport_ex3.dat
# ===========================================
# Data: 3 Warehouses -> 4 Customers
# ===========================================

set WAREHOUSES := W1 W2 W3;
set CUSTOMERS  := A B C D;

param supply :=
  W1 400
  W2 300
  W3 500;

param demand :=
  A 250
  B 350
  C 300
  D 200;

param cost :
      A   B   C   D :=
W1    8   6  10   9
W2    9  12  13   7
W3   14   9  16   5;

Writing transport_ex3.dat


In [16]:
from amplpy import AMPL

ex3 = AMPL()
ex3.read("transport_ex3.mod")
ex3.read_data("transport_ex3.dat")

ex3.option["solver"] = "highs"
ex3.solve()

print("Solver Status:", ex3.get_value("solve_result"))
print()

z = ex3.obj["Total_Shipping_Cost"].value()
print(f"Minimum Total Shipping Cost: ${z:,.2f}\n")

Ship = ex3.var["Ship"]

print("Optimal Shipping Plan:")
for w in ex3.set["WAREHOUSES"].members():
    for c in ex3.set["CUSTOMERS"].members():
        x = Ship[w,c].value()
        if x > 1e-6:
            print(f"  Ship[{w},{c}] = {x:.1f}")

print("\nSupply Slack + Duals (Supply_Limits):")
Supply_Limits = ex3.con["Supply_Limits"]
supply = ex3.param["supply"]

for w in ex3.set["WAREHOUSES"].members():
    used = Supply_Limits[w].body()
    cap = float(supply[w])
    slack = cap - used
    dual = Supply_Limits[w].dual()
    print(f"  {w}: Used {used:.1f}/{cap:.1f}, Slack {slack:.1f}, Dual {dual:.6f}")

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 9100
6 simplex iterations
0 barrier iterations
Solver Status: solved

Minimum Total Shipping Cost: $9,100.00

Optimal Shipping Plan:
  Ship[W1,B] = 100.0
  Ship[W1,C] = 300.0
  Ship[W2,A] = 250.0
  Ship[W3,B] = 250.0
  Ship[W3,D] = 200.0

Supply Slack + Duals (Supply_Limits):
  W1: Used 400.0/400.0, Slack 0.0, Dual -3.000000
  W2: Used 250.0/300.0, Slack 50.0, Dual 0.000000
  W3: Used 450.0/500.0, Slack 50.0, Dual 0.000000


Exercise 4 — Multi-Period Production Planning

In [17]:
%%writefile production_ex4.mod
# ===========================================
# Exercise 4: Multi-Period Production Planning
# ===========================================

set MONTHS ordered;

param demand {MONTHS} >= 0;
param prod_cost {MONTHS} >= 0;
param max_prod {MONTHS} >= 0;
param hold_cost {MONTHS} >= 0;

param init_inventory >= 0;
param final_inventory_min >= 0;

var Produce {m in MONTHS} >= 0, <= max_prod[m];
var Inventory {m in MONTHS} >= 0;

minimize Total_Cost:
    sum {m in MONTHS} (prod_cost[m] * Produce[m] + hold_cost[m] * Inventory[m]);

subject to Balance_First {m in MONTHS: ord(m)=1}:
    Inventory[m] = init_inventory + Produce[m] - demand[m];

subject to Balance_Rest {m in MONTHS: ord(m)>1}:
    Inventory[m] = Inventory[prev(m)] + Produce[m] - demand[m];

subject to Final_Inventory:
    Inventory[last(MONTHS)] >= final_inventory_min;

Writing production_ex4.mod


In [18]:
%%writefile production_ex4.dat
# ===========================================
# Data: 4-Month Planning
# ===========================================

set MONTHS := Jan Feb Mar Apr;

param demand :=
  Jan 100
  Feb 150
  Mar 130
  Apr 180;

param prod_cost :=
  Jan 20
  Feb 22
  Mar 21
  Apr 23;

param max_prod :=
  Jan 120
  Feb 120
  Mar 120
  Apr 120;

param hold_cost :=
  Jan 2
  Feb 2
  Mar 2
  Apr 2;

param init_inventory := 20;
param final_inventory_min := 10;

Writing production_ex4.dat


In [25]:
from amplpy import AMPL

ex4 = AMPL()
ex4.read("production_ex4.mod")
ex4.read_data("production_ex4.dat")

ex4.option["solver"] = "highs"
ex4.solve()

print("Solver Status:", ex4.get_value("solve_result"))
print("Solver Message:", ex4.get_value("solve_message"))
print()

# --- SAFE scalar reads ---
init_inv  = ex4.param["init_inventory"].value()
final_min = ex4.param["final_inventory_min"].value()

# --- Totals ---
MONTHS = list(ex4.set["MONTHS"].members())
demand_total = sum(ex4.param["demand"][m]   for m in MONTHS)
cap_total    = sum(ex4.param["max_prod"][m] for m in MONTHS)

required_prod = demand_total - init_inv + final_min
shortfall = required_prod - cap_total

print("Feasibility Check (Totals):")
print(f"  Total demand = {demand_total:.1f}")
print(f"  Total max production = {cap_total:.1f}")
print(f"  init_inventory = {init_inv:.1f}")
print(f"  final_inventory_min = {final_min:.1f}")
print(f"  Required total production = demand - init_inv + final_min = {required_prod:.1f}")
print(f"  Capacity shortfall = required - capacity = {shortfall:.1f}")
print()

jan = "Jan"
d_jan = ex4.param["demand"][jan]
cap_jan = ex4.param["max_prod"][jan]
rhs_max_Ijan = init_inv + cap_jan - d_jan

print("Presolve Contradiction (January):")
print("  Inventory[Jan] = init_inventory + Produce[Jan] - demand[Jan]")
print(f"  Produce[Jan] <= {cap_jan:.1f}, Inventory[Jan] >= 0")
print(f"  So Inventory[Jan] <= init_inv + max_prod[Jan] - demand[Jan] = {rhs_max_Ijan:.1f}")

if rhs_max_Ijan < 0:
    print(f"  But Inventory[Jan] must be >= 0, impossible since max possible is {rhs_max_Ijan:.1f}.")
    print(f"  The gap is {-rhs_max_Ijan:.1f} units (this matches the 'difference = 70' message).")

	presolve: constraint Balance_First['Jan'] cannot hold:
		body <= -80 cannot be >= -10; difference = -70


Solver Status: infeasible
Solver Message: presolve finds no feasible solution possible

Feasibility Check (Totals):
  Total demand = 560.0
  Total max production = 480.0
  init_inventory = 20.0
  final_inventory_min = 10.0
  Required total production = demand - init_inv + final_min = 550.0
  Capacity shortfall = required - capacity = 70.0

Presolve Contradiction (January):
  Inventory[Jan] = init_inventory + Produce[Jan] - demand[Jan]
  Produce[Jan] <= 120.0, Inventory[Jan] >= 0
  So Inventory[Jan] <= init_inv + max_prod[Jan] - demand[Jan] = 40.0


PART 2

Exercise 1 — Forest Service Allocation model

In [56]:
%%writefile forest.mod
set AREA;
set PRESC;

param s{AREA} >= 0;              # (000) acres
param p{AREA,PRESC};             # NPV per acre
param t{AREA,PRESC} >= 0;        # timber per (000) acres -> totals in (000) board-feet
param g{AREA,PRESC} >= 0;        # grazing per (000) acres -> totals in (000) AUM
param w{AREA,PRESC} >= 0;        # wilderness index

param TimberMin >= 0;
param GrazingMin >= 0;
param WildAvgMin >= 0;

param TotalAcres := sum{i in AREA} s[i];

var x{AREA,PRESC} >= 0;

maximize TotalNPV:
    sum{i in AREA, j in PRESC} p[i,j] * x[i,j];

subject to Allocate{i in AREA}:
    sum{j in PRESC} x[i,j] = s[i];

subject to TimberReq:
    sum{i in AREA, j in PRESC} t[i,j] * x[i,j] >= TimberMin;

subject to GrazingReq:
    sum{i in AREA, j in PRESC} g[i,j] * x[i,j] >= GrazingMin;

subject to WildernessReq:
    (1.0/TotalAcres) * sum{i in AREA, j in PRESC} w[i,j] * x[i,j] >= WildAvgMin;

Overwriting forest.mod


In [57]:
%%writefile forest.dat
data;

set AREA := 1 2 3 4 5 6 7;
set PRESC := 1 2 3;

param TimberMin := 40000;
param GrazingMin := 5;
param WildAvgMin := 70;

param s :=
1 75
2 90
3 140
4 60
5 212
6 98
7 113
;

param p:   1    2    3 :=
1        503  140  203
2        675  100   45
3        630  105   40
4        330   40  295
5        105  460  120
6        490   55  180
7        705   60  400
;

param t:   1    2    3 :=
1        310   50    0
2        198   46    0
3        210   57    0
4        112   30    0
5         40   32    0
6        105   25    0
7        213   40    0
;

param g:     1      2      3 :=
1          0.01   0.04   0
2          0.03   0.06   0
3          0.04   0.07   0
4          0.01   0.02   0
5          0.05   0.08   0
6          0.02   0.03   0
7          0.02   0.04   0
;

param w:   1    2    3 :=
1         40   80   95
2         55   60   65
3         45   55   60
4         30   35   90
5         60   60   70
6         35   50   75
7         40   45   95
;

end;

Overwriting forest.dat


In [58]:
from amplpy import AMPL

ampl = AMPL()
ampl.setOption("solver", "highs")

ampl.read("forest.mod")
ampl.readData("forest.dat")
ampl.solve()

print("solve_result  =", ampl.getValue("solve_result"))
print("solve_message =", ampl.getValue("solve_message"))

obj = ampl.getObjective("TotalNPV").value()
print("Total NPV (thousand $) =", obj)
print("Total NPV ($) =", obj * 1000)

ampl.eval("display {i in AREA, j in PRESC: x[i,j] > 1e-6} x[i,j];")

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 322515
17 simplex iterations
0 barrier iterations
solve_result  = solved
solve_message = HiGHS 1.11.0: optimal solution; objective 322515
17 simplex iterations
0 barrier iterations

Total NPV (thousand $) = 322515.00000000186
Total NPV ($) = 322515000.00000185
x[i,j] :=
1 3    75
2 1    90
3 1   140
4 3    60
5 2   154
5 3    58
6 3    98
7 3   113
;



In [61]:
# @title
import pandas as pd

# x is indexed by (AREA, PRESC)
x_df = ampl.getVariable("x").getValues().toPandas().reset_index()
# After reset_index, columns are typically: AREA, PRESC, x
x_df.columns = ["AREA", "PRESC", "x"]

# Show only nonzero allocations
x_nz = x_df[x_df["x"] > 1e-6].sort_values(["AREA", "PRESC"])
display(x_nz)

,AREA,PRESC,x
2,1,3,75.0
3,2,1,90.0
6,3,1,140.0
11,4,3,60.0
13,5,2,154.0
14,5,3,58.0
17,6,3,98.0
20,7,3,113.0


Exercise 2 — Swedish Steel Model

In [70]:
%%writefile swedish_steel.mod
# Swedish Steel Model (Application 4.2)

set ING;

param cost {ING} >= 0;     # kr/kg
param avail{ING} >= 0;     # kg available

param c  {ING} >= 0;       # carbon fraction
param ni {ING} >= 0;       # nickel fraction
param cr {ING} >= 0;       # chromium fraction
param mo {ING} >= 0;       # molybdenum fraction

param W := 1000;

param Cmin;  param Cmax;
param NImin; param NImax;
param CRmin; param CRmax;
param MOmin; param MOmax;

var x {j in ING} >= 0, <= avail[j];

minimize TotalCost:
    sum {j in ING} cost[j] * x[j];

subject to TotalWeight:
    sum {j in ING} x[j] = W;

subject to CarbonMin:
    sum {j in ING} c[j] * x[j] >= Cmin * W;

subject to CarbonMax:
    sum {j in ING} c[j] * x[j] <= Cmax * W;

subject to NickelMin:
    sum {j in ING} ni[j] * x[j] >= NImin * W;

subject to NickelMax:
    sum {j in ING} ni[j] * x[j] <= NImax * W;

subject to ChromiumMin:
    sum {j in ING} cr[j] * x[j] >= CRmin * W;

subject to ChromiumMax:
    sum {j in ING} cr[j] * x[j] <= CRmax * W;

subject to MolyMin:
    sum {j in ING} mo[j] * x[j] >= MOmin * W;

subject to MolyMax:
    sum {j in ING} mo[j] * x[j] <= MOmax * W;

Overwriting swedish_steel.mod


In [71]:
%%writefile swedish_steel.dat
data;

set ING := scrap1 scrap2 scrap3 scrap4 nickel chromium moly;

param: cost avail c      ni      cr      mo :=
scrap1    16    75   0.0080  0.180   0.120   0.0000
scrap2    10    250  0.0070  0.032   0.011   0.0010
scrap3     8    1e9  0.0085  0.000   0.000   0.0000
scrap4     9    1e9  0.0040  0.000   0.000   0.0000
nickel    48    1e9  0.0000  1.000   0.000   0.0000
chromium  60    1e9  0.0000  0.000   1.000   0.0000
moly      53    1e9  0.0000  0.000   0.000   1.0000
;

param Cmin  := 0.0065;  param Cmax  := 0.0075;
param NImin := 0.0300;  param NImax := 0.0350;
param CRmin := 0.0100;  param CRmax := 0.0120;
param MOmin := 0.0110;  param MOmax := 0.0130;

end;

Overwriting swedish_steel.dat


In [72]:
ampl = AMPL()
ampl.setOption("solver", "highs")

ampl.read("swedish_steel.mod")
ampl.readData("swedish_steel.dat")
ampl.solve()

print("solve_result :", ampl.getValue("solve_result"))
print("solve_message:", ampl.getValue("solve_message"))

obj = ampl.getObjective("TotalCost").value()
print("\nOptimal cost (kr) =", obj)

# ---- solution into pandas ----
x_df = ampl.getVariable("x").getValues().toPandas().reset_index()
x_df.columns = ["ING", "kg"]
x_df = x_df.sort_values("kg", ascending=False)
display(x_df)

# ---- compare against textbook reported optimum ----
book = {
    "scrap1": 75.00,
    "scrap2": 90.91,
    "scrap3": 672.28,
    "scrap4": 137.31,
    "nickel": 13.59,
    "chromium": 0.00,
    "moly": 10.91
}
book_df = pd.DataFrame({"ING": list(book.keys()), "book_kg": list(book.values())})

chk = x_df.merge(book_df, on="ING", how="left").fillna(0)
chk["diff"] = chk["kg"] - chk["book_kg"]
display(chk)

print("Max |diff| =", chk["diff"].abs().max())

# ---- composition check (in %) ----
c  = ampl.getParameter("c").getValues().toPandas().reset_index();  c.columns  = ["ING","c"]
ni = ampl.getParameter("ni").getValues().toPandas().reset_index(); ni.columns = ["ING","ni"]
cr = ampl.getParameter("cr").getValues().toPandas().reset_index(); cr.columns = ["ING","cr"]
mo = ampl.getParameter("mo").getValues().toPandas().reset_index(); mo.columns = ["ING","mo"]

df = x_df.merge(c,on="ING").merge(ni,on="ING").merge(cr,on="ING").merge(mo,on="ING")

W = 1000.0
C_pct  = 100*(df["c"] *df["kg"]).sum()/W
NI_pct = 100*(df["ni"]*df["kg"]).sum()/W
CR_pct = 100*(df["cr"]*df["kg"]).sum()/W
MO_pct = 100*(df["mo"]*df["kg"]).sum()/W

print("\nBlend composition (%)")
print("Carbon     =", C_pct)
print("Nickel     =", NI_pct)
print("Chromium   =", CR_pct)
print("Molybdenum =", MO_pct)

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 9953.671717
5 simplex iterations
0 barrier iterations
solve_result : solved
solve_message: HiGHS 1.11.0: optimal solution; objective 9953.671717
5 simplex iterations
0 barrier iterations


Optimal cost (kr) = 9953.671717171715


,ING,kg
5,scrap3,672.282828
6,scrap4,137.308081
4,scrap2,90.909091
3,scrap1,75.000000
2,nickel,13.590909
1,moly,10.909091
0,chromium,0.000000


,ING,kg,book_kg,diff
0,scrap3,672.282828,672.28,0.002828
1,scrap4,137.308081,137.31,-0.001919
2,scrap2,90.909091,90.91,-0.000909
3,scrap1,75.000000,75.00,0.000000
4,nickel,13.590909,13.59,0.000909
5,moly,10.909091,10.91,-0.000909
6,chromium,0.000000,0.00,0.000000


Max |diff| = 0.0028282828282044647

Blend composition (%)
Carbon     = 0.75
Nickel     = 3.0
Chromium   = 1.0
Molybdenum = 1.1000000000000003


Exercise 3 — Tubular Products Operations Planning model

In [73]:
%%writefile tubular.mod
# Tubular Products Operations Planning (Application 4.3)

set PROD;
set MILL;
set ALLOW within (PROD cross MILL);

param demand {PROD} >= 0;     # (1000 lb/week)
param cap    {MILL} >= 0;     # (hours/week)

param cost {ALLOW} >= 0;      # $ per (1000 lb)
param time {ALLOW} >= 0;      # hours per (1000 lb)

var x {ALLOW} >= 0;           # (1000 lb/week)

minimize TotalCost:
    sum {(p,m) in ALLOW} cost[p,m] * x[p,m];

subject to MeetDemand {p in PROD}:
    sum {m in MILL: (p,m) in ALLOW} x[p,m] >= demand[p];

subject to MillCap {m in MILL}:
    sum {p in PROD: (p,m) in ALLOW} time[p,m] * x[p,m] <= cap[m];

Writing tubular.mod


In [74]:
%%writefile tubular.dat
data;

set PROD := 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16;
set MILL := 1 2 3 4;

param demand :=
1 100
2 630
3 500
4 980
5 720
6 240
7 75
8 22
9 50
10 22
11 353
12 55
13 125
14 35
15 100
16 10
;

param cap :=
1 800
2 480
3 1280
4 960
;

set ALLOW :=
(1,1) (1,2) (1,3) (1,4)
(2,1) (2,2) (2,3) (2,4)
(3,1) (3,2) (3,3) (3,4)
(4,1) (4,2) (4,3) (4,4)
(5,1) (5,2) (5,3) (5,4)
(6,1) (6,2) (6,3) (6,4)
(7,2) (7,3) (7,4)
(8,2) (8,3) (8,4)
(9,1) (9,2) (9,4)
(10,1) (10,2) (10,4)
(11,1) (11,2) (11,4)
(12,1) (12,2) (12,4)
(13,1) (13,2) (13,4)
(14,1) (14,2) (14,4)
(15,2) (15,4)
(16,2) (16,4)
;

param cost :=
[1,1] 90   [1,2] 75   [1,3] 70   [1,4] 63
[2,1] 80   [2,2] 70   [2,3] 65   [2,4] 60
[3,1] 104  [3,2] 85   [3,3] 83   [3,4] 77
[4,1] 98   [4,2] 79   [4,3] 80   [4,4] 74
[5,1] 123  [5,2] 101  [5,3] 110  [5,4] 99
[6,1] 113  [6,2] 94   [6,3] 100  [6,4] 84
[7,2] 160  [7,3] 156  [7,4] 140
[8,2] 142  [8,3] 150  [8,4] 130
[9,1] 140  [9,2] 110  [9,4] 122
[10,1] 124 [10,2] 96  [10,4] 101
[11,1] 160 [11,2] 133 [11,4] 138
[12,1] 143 [12,2] 127 [12,4] 133
[13,1] 202 [13,2] 150 [13,4] 160
[14,1] 190 [14,2] 141 [14,4] 140
[15,2] 190 [15,4] 220
[16,2] 175 [16,4] 200
;

param time :=
[1,1] 0.8  [1,2] 0.7  [1,3] 0.5  [1,4] 0.6
[2,1] 0.8  [2,2] 0.7  [2,3] 0.5  [2,4] 0.6
[3,1] 0.8  [3,2] 0.7  [3,3] 0.5  [3,4] 0.6
[4,1] 0.8  [4,2] 0.7  [4,3] 0.5  [4,4] 0.6
[5,1] 0.8  [5,2] 0.7  [5,3] 0.5  [5,4] 0.6
[6,1] 0.8  [6,2] 0.7  [6,3] 0.5  [6,4] 0.6
[7,2] 0.9  [7,3] 0.5  [7,4] 0.6
[8,2] 0.9  [8,3] 0.5  [8,4] 0.6
[9,1] 1.5  [9,2] 0.9  [9,4] 1.2
[10,1] 1.5 [10,2] 0.9 [10,4] 1.2
[11,1] 1.5 [11,2] 0.9 [11,4] 1.2
[12,1] 1.5 [12,2] 0.9 [12,4] 1.2
[13,1] 1.5 [13,2] 0.9 [13,4] 1.2
[14,1] 1.5 [14,2] 0.9 [14,4] 1.2
[15,2] 1.0 [15,4] 1.5
[16,2] 1.0 [16,4] 1.5
;

end;

Writing tubular.dat


In [75]:
ampl = AMPL()
ampl.setOption("solver", "highs")

ampl.read("tubular.mod")
ampl.readData("tubular.dat")
ampl.solve()

print("solve_result :", ampl.getValue("solve_result"))
print("solve_message:", ampl.getValue("solve_message"))

obj = ampl.getObjective("TotalCost").value()
print("\nOptimal total weekly cost ($) =", obj)

# ---- x into pandas ----
x_df = ampl.getVariable("x").getValues().toPandas().reset_index()
x_df.columns = ["PROD", "MILL", "x"]
x_df["PROD"] = x_df["PROD"].astype(int)
x_df["MILL"] = x_df["MILL"].astype(int)

# Pivot (products x mills)
x_pivot = x_df.pivot(index="PROD", columns="MILL", values="x").fillna(0).sort_index()
display(x_pivot)

# Nonzero only
x_nz = x_df[x_df["x"].abs() > 1e-6].sort_values(["PROD","MILL"])
display(x_nz)

# ---- Demand satisfaction check ----
demand = ampl.getParameter("demand").getValues().toPandas().reset_index()
demand.columns = ["PROD", "demand"]
demand["PROD"] = demand["PROD"].astype(int)

prod_total = x_df.groupby("PROD", as_index=False)["x"].sum().rename(columns={"x":"total_prod"})
dem_chk = demand.merge(prod_total, on="PROD")
dem_chk["slack (total - demand)"] = dem_chk["total_prod"] - dem_chk["demand"]
display(dem_chk.sort_values("PROD"))

# ---- Capacity utilisation check ----
time_df = ampl.getParameter("time").getValues().toPandas().reset_index()
time_df.columns = ["PROD","MILL","time"]
time_df["PROD"] = time_df["PROD"].astype(int)
time_df["MILL"] = time_df["MILL"].astype(int)

cap = ampl.getParameter("cap").getValues().toPandas().reset_index()
cap.columns = ["MILL","cap"]
cap["MILL"] = cap["MILL"].astype(int)

use = x_df.merge(time_df, on=["PROD","MILL"])
use["hours_used"] = use["x"] * use["time"]
mill_use = use.groupby("MILL", as_index=False)["hours_used"].sum().merge(cap, on="MILL")
mill_use["slack (cap - used)"] = mill_use["cap"] - mill_use["hours_used"]
display(mill_use.sort_values("MILL"))

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 378899.1111
22 simplex iterations
0 barrier iterations
solve_result : solved
solve_message: HiGHS 1.11.0: optimal solution; objective 378899.1111
22 simplex iterations
0 barrier iterations


Optimal total weekly cost ($) = 378899.11111111107


MILL,1,2,3,4
PROD,,,,
1,0.0,0.000000,0.000000,100.000000
2,0.0,0.000000,630.000000,0.000000
3,0.0,0.000000,500.000000,0.000000
4,0.0,0.000000,884.777778,95.222222
5,0.0,0.000000,0.000000,720.000000
6,0.0,0.000000,0.000000,240.000000
7,0.0,0.000000,0.000000,75.000000
8,0.0,0.000000,0.000000,22.000000
9,0.0,50.000000,0.000000,0.000000


,PROD,MILL,x
3,1,4,100.000000
6,2,3,630.000000
10,3,3,500.000000
14,4,3,884.777778
15,4,4,95.222222
19,5,4,720.000000
23,6,4,240.000000
26,7,4,75.000000
29,8,4,22.000000
31,9,2,50.000000


,PROD,demand,total_prod,slack (total - demand)
0,1,100,100.0,0.000000e+00
1,2,630,630.0,-1.136868e-13
2,3,500,500.0,1.136868e-13
3,4,980,980.0,0.000000e+00
4,5,720,720.0,0.000000e+00
5,6,240,240.0,0.000000e+00
6,7,75,75.0,0.000000e+00
7,8,22,22.0,0.000000e+00
8,9,50,50.0,0.000000e+00
9,10,22,22.0,0.000000e+00


,MILL,hours_used,cap,slack (cap - used)
0,1,82.500000,800,717.500000
1,2,480.000000,480,0.000000
2,3,1007.388889,1280,272.611111
3,4,960.000000,960,0.000000


Exercise 4 — Ohio National Bank model

In [76]:
%%writefile onb.mod
# Ohio National Bank (ONB) Shift Scheduling — Application 4.5
# Units:
# arrivals and backlog are in THOUSANDS of checks.
# Full-time processes 1 (thousand checks)/hour.
# Part-time processes 0.8 (thousand checks)/hour.

set HT;          # hours in the planning horizon (11..21)
set HFT;         # full-time shift start times (11,12,13)
set HOT;         # overtime start times (11,12)
set HPT;         # part-time shift start times (11..18)
set HW;          # backlog hours (12..21)

param machine_max >= 0;
param overtime_max >= 0;
param backlog_max  >= 0;

param arr {HT} >= 0;          # arrivals (thousand checks)

param ft_rate >= 0;
param pt_rate >= 0;

param cost_x {HFT} >= 0;
param cost_y {HOT} >= 0;
param cost_z {HPT} >= 0;

# Who is on duty in each hour (from Table 4.6 / Table 4.7)
set FT_ON {HT} within HFT;
set OT_ON {HT} within HOT;
set PT_ON {HT} within HPT;

var x {h in HFT} >= 0;                       # full-time starting at h
var y {h in HOT} >= 0;                       # overtime workers from shift h
var z {h in HPT} >= 0;                       # part-time starting at h
var w {h in HW}  >= 0, <= backlog_max;       # backlog at hour h (thousand checks)

minimize TotalPay:
    sum {h in HFT} cost_x[h] * x[h]
  + sum {h in HOT} cost_y[h] * y[h]
  + sum {h in HPT} cost_z[h] * z[h];

# Machine capacity: total workers on duty each hour <= machine_max
subject to MachineCap {t in HT}:
    sum {h in FT_ON[t]} x[h]
  + sum {h in OT_ON[t]} y[h]
  + sum {h in PT_ON[t]} z[h]
  <= machine_max;

# Overtime limits
subject to OT_limit_11: y[11] <= 0.5 * x[11];
subject to OT_limit_12: y[12] <= 0.5 * x[12];
subject to OT_total:    y[11] + y[12] <= overtime_max;

# Processing (cover) constraints with backlog flow
# Interpretation: processing in hour t must cover arrivals in hour t,
# allowing backlog to carry forward (w[t+1]) and accounting for existing backlog (w[t]).

# Hour 11: w[11] = 0, so proc11 >= arr11 - w12
subject to Cover11:
    ft_rate * (sum {h in FT_ON[11]} x[h] + sum {h in OT_ON[11]} y[h])
  + pt_rate *  sum {h in PT_ON[11]} z[h]
  >= arr[11] - w[12];

# Hours 12..20: proc[t] >= arr[t] + w[t] - w[t+1]
subject to CoverMid {t in HT: t >= 12 && t <= 20}:
    ft_rate * (sum {h in FT_ON[t]} x[h] + sum {h in OT_ON[t]} y[h])
  + pt_rate *  sum {h in PT_ON[t]} z[h]
  >= arr[t] + w[t] - w[t+1];

# Hour 21: must finish by 22:00, so w[22]=0 -> proc21 >= arr21 + w21
subject to Cover21:
    ft_rate * (sum {h in FT_ON[21]} x[h] + sum {h in OT_ON[21]} y[h])
  + pt_rate *  sum {h in PT_ON[21]} z[h]
  >= arr[21] + w[21];

Writing onb.mod


In [77]:
%%writefile onb.dat
data;

set HT  := 11 12 13 14 15 16 17 18 19 20 21;
set HFT := 11 12 13;
set HOT := 11 12;
set HPT := 11 12 13 14 15 16 17 18;
set HW  := 12 13 14 15 16 17 18 19 20 21;

param machine_max := 35;
param overtime_max := 20;
param backlog_max := 20;

param ft_rate := 1.0;
param pt_rate := 0.8;

# Arrivals (thousand checks)
param arr :=
11 10
12 11
13 15
14 20
15 25
16 28
17 32
18 50
19 30
20 20
21 8
;

# Objective coefficients from Table 4.7
param cost_x :=
11 90
12 91
13 92
;

param cost_y :=
11 18
12 18
;

param cost_z :=
11 28
12 28
13 28
14 28
15 29
16 30
17 31
18 32
;

# On-duty sets by hour (from Table 4.6 / Table 4.7)
set FT_ON[11] := 11;
set FT_ON[12] := 11 12;
set FT_ON[13] := 11 12 13;
set FT_ON[14] := 11 12 13;
set FT_ON[15] := 12 13;
set FT_ON[16] := 11 13;
set FT_ON[17] := 11 12;
set FT_ON[18] := 11 12 13;
set FT_ON[19] := 11 12 13;
set FT_ON[20] := 12 13;
set FT_ON[21] := 13;

set OT_ON[11] := ;
set OT_ON[12] := ;
set OT_ON[13] := ;
set OT_ON[14] := ;
set OT_ON[15] := ;
set OT_ON[16] := ;
set OT_ON[17] := ;
set OT_ON[18] := ;
set OT_ON[19] := ;
set OT_ON[20] := 11;
set OT_ON[21] := 12;

set PT_ON[11] := 11;
set PT_ON[12] := 11 12;
set PT_ON[13] := 11 12 13;
set PT_ON[14] := 11 12 13 14;
set PT_ON[15] := 12 13 14 15;
set PT_ON[16] := 13 14 15 16;
set PT_ON[17] := 14 15 16 17;
set PT_ON[18] := 15 16 17 18;
set PT_ON[19] := 16 17 18;
set PT_ON[20] := 17 18;
set PT_ON[21] := 18;

end;

Writing onb.dat


In [79]:
ampl = AMPL()
ampl.setOption("solver", "highs")

ampl.read("onb.mod")
ampl.readData("onb.dat")
ampl.solve()

print("solve_result :", ampl.getValue("solve_result"))
print("solve_message:", ampl.getValue("solve_message"))

obj = ampl.getObjective("TotalPay").value()
print("\nOptimal total pay ($) =", obj)

# --- Pull variables into pandas ---
x_df = ampl.getVariable("x").getValues().toPandas().reset_index()
x_df.columns = ["start_h", "x_fulltime"]

y_df = ampl.getVariable("y").getValues().toPandas().reset_index()
y_df.columns = ["start_h", "y_overtime"]

z_df = ampl.getVariable("z").getValues().toPandas().reset_index()
z_df.columns = ["start_h", "z_parttime"]

w_df = ampl.getVariable("w").getValues().toPandas().reset_index()
w_df.columns = ["hour", "backlog_w"]

display(x_df)
display(y_df)
display(z_df[z_df["z_parttime"].abs() > 1e-8].sort_values("start_h"))
display(w_df[w_df["backlog_w"].abs() > 1e-8].sort_values("hour"))

# --- Recreate the on-duty mappings (same as data) for checks ---
HT = list(range(11, 22))
FT_ON = {
    11:[11], 12:[11,12], 13:[11,12,13], 14:[11,12,13], 15:[12,13],
    16:[11,13], 17:[11,12], 18:[11,12,13], 19:[11,12,13], 20:[12,13], 21:[13]
}
OT_ON = {t:[] for t in HT}
OT_ON[20] = [11]
OT_ON[21] = [12]
PT_ON = {
    11:[11], 12:[11,12], 13:[11,12,13], 14:[11,12,13,14], 15:[12,13,14,15],
    16:[13,14,15,16], 17:[14,15,16,17], 18:[15,16,17,18], 19:[16,17,18],
    20:[17,18], 21:[18]
}

x = dict(zip(x_df["start_h"].astype(int), x_df["x_fulltime"].astype(float)))
y = dict(zip(y_df["start_h"].astype(int), y_df["y_overtime"].astype(float)))
z = dict(zip(z_df["start_h"].astype(int), z_df["z_parttime"].astype(float)))

machine_max = float(ampl.getParameter("machine_max").value())
ft_rate = float(ampl.getParameter("ft_rate").value())
pt_rate = float(ampl.getParameter("pt_rate").value())

# --- Machine capacity check ---
rows = []
for t in HT:
    on_duty = sum(x.get(h,0.0) for h in FT_ON[t]) + sum(y.get(h,0.0) for h in OT_ON[t]) + sum(z.get(h,0.0) for h in PT_ON[t])
    rows.append([t, on_duty, machine_max - on_duty])
mach = pd.DataFrame(rows, columns=["hour","workers_on_duty","slack (35 - on_duty)"])
display(mach)

# --- Backlog flow check (FIXED: no index0 anywhere) ---
arr_df = ampl.getParameter("arr").getValues().toPandas()
arr = arr_df.iloc[:, 0].astype(float).to_dict()

w = dict(zip(w_df["hour"].astype(int), w_df["backlog_w"].astype(float)))

def proc(t):
    return ft_rate*(sum(x.get(h,0.0) for h in FT_ON[t]) + sum(y.get(h,0.0) for h in OT_ON[t])) \
         + pt_rate*sum(z.get(h,0.0) for h in PT_ON[t])

flow = []
flow.append([11, arr[11], proc(11), 0.0, w.get(12, 0.0)])  # w11 = 0

for t in range(12, 21):
    flow.append([t, arr[t], proc(t), w.get(t, 0.0), w.get(t+1, 0.0)])

# hour 21 ends with w22=0
flow.append([21, arr[21], proc(21), w.get(21, 0.0), 0.0])

flow_df = pd.DataFrame(flow, columns=["hour","arrivals","processing","w_t","w_next"])
flow_df["balance (w_t + arrivals - processing - w_next)"] = (
    flow_df["w_t"] + flow_df["arrivals"] - flow_df["processing"] - flow_df["w_next"]
)
display(flow_df)
print("Max absolute balance error:", flow_df["balance (w_t + arrivals - processing - w_next)"].abs().max())

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 2836.071429
29 simplex iterations
0 barrier iterations
solve_result : solved
solve_message: HiGHS 1.11.0: optimal solution; objective 2836.071429
29 simplex iterations
0 barrier iterations


Optimal total pay ($) = 2836.071428571418


,start_h,x_fulltime
0,11,5.050319e-13
1,12,8.571429e+00
2,13,1.285714e+01


,start_h,y_overtime
0,11,2.642331e-13
1,12,4.285714e+00


,start_h,z_parttime
3,14,13.571429
5,16,5.357143
6,17,7.500000
7,18,0.714286


,hour,backlog_w
0,12,10.000000
1,13,12.428571
2,14,6.000000
6,18,2.285714
7,19,20.000000
8,20,17.714286
9,21,9.714286


,hour,workers_on_duty,slack (35 - on_duty)
0,11,5.050319e-13,3.500000e+01
1,12,8.571429e+00,2.642857e+01
2,13,2.142857e+01,1.357143e+01
3,14,3.500000e+01,-4.476419e-13
4,15,3.500000e+01,8.526513e-14
5,16,3.178571e+01,3.214286e+00
6,17,3.500000e+01,1.847411e-13
7,18,3.500000e+01,2.842171e-14
8,19,3.500000e+01,-7.105427e-15
9,20,2.964286e+01,5.357143e+00


,hour,arrivals,processing,w_t,w_next,balance (w_t + arrivals - processing - w_next)
0,11,10.0,5.050319e-13,0.000000,10.000000,0.000000e+00
1,12,11.0,8.571429e+00,10.000000,12.428571,-1.776357e-15
2,13,15.0,2.142857e+01,12.428571,6.000000,-1.776357e-15
3,14,20.0,3.228571e+01,6.000000,0.000000,-6.285714e+00
4,15,25.0,3.228571e+01,0.000000,0.000000,-7.285714e+00
5,16,28.0,2.800000e+01,0.000000,0.000000,1.492140e-13
6,17,32.0,2.971429e+01,0.000000,2.285714,1.412204e-13
7,18,50.0,3.228571e+01,2.285714,20.000000,-7.105427e-15
8,19,30.0,3.228571e+01,20.000000,17.714286,1.065814e-14
9,20,20.0,2.800000e+01,17.714286,9.714286,-1.723066e-13


Max absolute balance error: 7.285714285713894
